In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
class RubanConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        super(RubanConv2D, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride if isinstance(stride, tuple) else (stride, stride)
        self.padding = padding if isinstance(padding, tuple) else (padding, padding)

        self.bias = bias
        self.weight = nn.Parameter(
            torch.randn(out_channels, in_channels, *self.kernel_size) * np.sqrt(2.0 / (in_channels * np.prod(self.kernel_size)))
        )
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_channels))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        if any(self.padding):
            x = F.pad(x, (self.padding[1], self.padding[1], self.padding[0], self.padding[0]), mode='constant', value=0)

        batch_size, _, input_height, input_width = x.shape
        output_height = (input_height - self.kernel_size[0]) // self.stride[0] + 1
        output_width = (input_width - self.kernel_size[1]) // self.stride[1] + 1

        output = torch.zeros(batch_size, self.out_channels, output_height, output_width, device=x.device)

        for i in range(0, output_height):
            for j in range(0, output_width):
                region = x[:, :, i * self.stride[0]:i * self.stride[0] + self.kernel_size[0], j * self.stride[1]:j * self.stride[1] + self.kernel_size[1]]
                output[:, :, i, j] = torch.sum(region.unsqueeze(1) * self.weight, dim=(2, 3, 4))

        if self.bias is not None:
            output += self.bias.view(1, -1, 1, 1)

        return output